In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.callbacks import LearningRateScheduler
import keras.backend as K
from tensorflow.keras.metrics import RootMeanSquaredError as rmse


In [ ]:
def MOUNT():
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
def NewTimeStampDataframe(df,time):
  df.rename(columns={'Days' : 'timestamp', 'Open' : 'open', 
                    'High' : 'high', 'Low' : 'low', 'Close' : 'close'}, inplace=True)
  df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)
  df.set_index('timestamp', inplace=True)
 
  df = df.groupby(pd.Grouper(freq=time)).agg({"open": "first", 
                                              "close": "last", 
                                              "low": "min", 
                                              "high": "max"})
  df.columns = ["open", "close", "low", "high"]
  return df

In [ ]:
def AddingColumn(df):
  df['previous_close'] = df['close'].shift(-1)
  df['hour'] = df.index.hour
  df['day']  = df.index.weekday
  df['week'] = df.index.week
  #df['dayofweek']=df.index.dayofweek
  #df['quarter']=df.index.quarter
  #df['month']=df.index.month
  #df['year']=df.index.year
  #df['dayofyear']=df.index.dayofyear
  
  df['momentum']  = (df['open'] - df['close'])
  df['avg_price'] = (df['low'] + df['high'])/2
  df['range']     = df['high'] - df['low']
  df['ohlc_price'] = (df['low'] + df['high'] + df['open'] + df['close'])/4
  df.dropna(inplace=True)
  return df

In [ ]:
def ApplyPCA(ndf):
  from sklearn.decomposition import PCA

  Dataset = ndf.copy().values.astype('float32') # D_S = Dataset
  pca_features = ndf.columns.tolist()

  pca = PCA(n_components=1)
  ndf['pca'] = pca.fit_transform(Dataset)
  return ndf

In [ ]:
def df_to_numpy_array_features(x_train, x_val, x_test, window_size):
  
  xtrain = []
  xval = []
  xtest = []

  for i in range(len(x_train)-window_size-1):
    
    train_row = x_train[i:i+window_size]
   
    xtrain.append(train_row)
  
  for i in range(len(x_val)-window_size-1):
    
    val_row = x_val[i:i+window_size]
   
    xval.append(val_row)

  for i in range(len(x_test)-window_size-1):
    
    test_row = x_test[i:i+window_size]
   
    xtest.append(test_row)
  return np.array(xtrain), np.array(xval), np.array(xtest)

In [ ]:
def df_to_numpy_array_target(y_train, y_val, y_test, window_size):
  
  
  ytrain = []
  yval = []
  ytest = [] 
  for i in range(len(y_train)-window_size-1):
    
    train_label = y_train[i+window_size]
    ytrain.append(train_label)

  for i in range(len(y_val)-window_size-1):
    
    val_label = y_val[i+window_size]
    yval.append(val_label)

  for i in range(len(y_test)-window_size-1):
    
    test_label = y_test[i+window_size]
    ytest.append(test_label)
    
  return  np.array(ytrain), np.array(yval), np.array(ytest)

In [ ]:
def df_to_X_y(df, window_size):
  
  X = []
  y = []
  for i in range(len(df)-window_size-1):
    row = df[i:i+window_size]
    X.append(row)
    label = df[i+window_size]
    y.append(label)
    
  return np.array(X), np.array(y)

In [ ]:
def NormalizeDataset(Dataset):
  #print("Before Normalization")
  #print("Min:", np.min(Dataset))
  #print("Max:", np.max(Dataset))
  scaler = MinMaxScaler(feature_range=(0, 1))
  Dataset = scaler.fit_transform(Dataset)
  #print("After Normalization")
  #print("Min:", np.min(Dataset))
  #print("Max:", np.max(Dataset))
  return Dataset

In [ ]:
def Train_Val_Test_split_60_40(X1,y1):
  # Define the split time
  train_split = int(len(X1) * 0.60)
  test_split = int(len(X1) * 0.20)

  # Get the train set 
  x_train = X1[:train_split]
  y_train = y1[:train_split]
  x_val= X1[train_split:(train_split + test_split)]
  y_val= y1[train_split:(train_split + test_split)]
            
  # Get the validation set
  x_test = X1[(train_split + test_split):]
  y_test = y1[(train_split + test_split):]
  print(len(x_train))
  print(len(y_train))
  print(len(x_val))
  print(len(y_val))
  print(len(x_test))
  print(len(y_test))
  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def Train_Val_Test_split_70_30(X1,y1):
  # Define the split time
  train_split = int(len(X1) * 0.70)
  test_split = int(len(X1) * 0.15)

  # Get the train set 
  x_train = X1[:train_split]
  y_train = y1[:train_split]
  x_val= X1[train_split:(train_split + test_split)]
  y_val= y1[train_split:(train_split + test_split)]
            
  # Get the validation set
  x_test = X1[(train_split + test_split):]
  y_test = y1[(train_split + test_split):]
  print(len(x_train))
  print(len(y_train))
  print(len(x_val))
  print(len(y_val))
  print(len(x_test))
  print(len(y_test))
  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def Train_Val_Test_split_80_20(X1,y1):
  # Define the split time
  train_split = int(len(X1) * 0.80)
  test_split = int(len(X1) * 0.20)

  # Get the train set 
  x_train = X1[:train_split]
  y_train = y1[:train_split]
  x_val= X1[train_split:(train_split + test_split)]
  y_val= y1[train_split:(train_split + test_split)]
            
  # Get the validation set
  x_test = X1[(train_split + test_split):]
  y_test = y1[(train_split + test_split):]
  print(len(x_train))
  print(len(y_train))
  print(len(x_val))
  print(len(y_val))
  print(len(x_test))
  print(len(y_test))
  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def Train_Val_Test_split_15Min(X1,y1):
  # Define the split time
  split_time = int(len(X1) * 0.80)

  # Get the train set 
  x_train = X1[:split_time]
  y_train = y1[:split_time]
  x_val= X1[split_time:(split_time+7500)]
  y_val= y1[split_time:(split_time+7500)]
            
  # Get the validation set
  x_test = X1[(split_time+7500):]
  y_test = y1[(split_time+7500):]
  print(len(x_train))
  print(len(y_train))
  print(len(x_val))
  print(len(y_val))
  print(len(x_test))
  print(len(y_test))
  return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
def CNN_LSTM_Model(X1):
    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                        strides=1,
                        activation="relu",
                        padding='causal',
                        input_shape=[X1.shape[1], X1.shape[2]]),
    

    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
    ])
    return model

In [ ]:
def CreateHistory(x_train, y_train, x_val, y_val, model, callbacks_list, Epochs, Batch_size):

  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse', rmse(),'mape'])
  history = model.fit(x_train,y_train, epochs=Epochs, batch_size=Batch_size, verbose=1, callbacks=callbacks_list, validation_data=(x_val, y_val))
  return history

In [ ]:
def R_Squared_Score(model, x, y):
  from sklearn.metrics import r2_score as r2
  y_pred = model.predict(x)

  # Compute the R^2 score
  r_squared = r2(y, y_pred)
  return r_squared

In [ ]:
def SummarizeHistory(history):  
  epoch = len(history.history['loss'])
  for k in list(history.history.keys()):
      if 'val' not in k:
        plt.figure(figsize=(40,10))
        plt.plot(history.history[k])
        #plt.plot(history.history['val_' + k])
        plt.title(k)
        plt.ylabel(k)
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()

In [ ]:
def scheduler(model, epoch):
    from keras.callbacks import LearningRateScheduler
    import keras.backend as K
    if epoch%2==0 and epoch!=0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr*.9)
        print("lr changed to {}".format(lr*.9))
    return K.get_value(model.optimizer.lr)

In [ ]:
def CreatPrediction(model,x, y):
  pred = model.predict(x)

  predictions = pd.DataFrame()
  predictions['predicted'] = pd.Series(np.reshape(pred, (pred.shape[0])))
  predictions['actual'] = y
  predictions = predictions.astype(float)
  predictions.plot(figsize=(20,10))
  plt.show()
  return predictions

In [ ]:
def GRU_LSTM_Model(X1):
  model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32,
                        input_shape=[X1.shape[1], X1.shape[2]],return_sequences=True),
    tf.keras.layers.LSTM(256, return_sequences=True),
    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(64),
    
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
  ])
  return model

In [ ]:
def CNN_GRU_Model(X1):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape=[X1.shape[1], X1.shape[2]]),
    tf.keras.layers.GRU(256, return_sequences=True),
    
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(64),

    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
  ])
  return model


In [ ]:
def GRU_Model(X1):
  model = tf.keras.models.Sequential([
                      
    tf.keras.layers.GRU(256, input_shape=[X1.shape[1], X1.shape[2]], return_sequences=True),
    
    tf.keras.layers.GRU(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(16),

    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
  ])
  return model

In [ ]:
def LSTM_Model(X1):
  model = tf.keras.models.Sequential([
                      
    tf.keras.layers.LSTM(256, input_shape=[X1.shape[1], X1.shape[2]], return_sequences=True),
    
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(16),

    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
  ])
  return model

In [ ]:
def CNN_BiLSTM_Model(X1):

  model= tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape=[X1.shape[1], X1.shape[2]]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
    
    ])
  return model

In [ ]:
def GRU_BiLSTM_Model(X1):

  model= tf.keras.models.Sequential([
    tf.keras.layers.GRU(32,
                        input_shape=[X1.shape[1], X1.shape[2]],return_sequences=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
    
    ])
  return model

In [ ]:
def BiLSTM_Model(X1):

  model= tf.keras.models.Sequential([

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, input_shape=[X1.shape[1], X1.shape[2]], return_sequences=True)),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
    
    ])
  return model

In [ ]:
def CNN_Model(X1):

  model= tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                      strides=1,
                      activation="relu",
                      padding='causal',
                      input_shape=[X1.shape[1], X1.shape[2]]),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Lambda(lambda x: x * 400)
    ])
  return model